In [ ]:
# Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import math
import scipy.stats as ss
import random
import time

In [ ]:
# Set parameters
h = 50 # number of runs of simulation
m = 1000 # number of simulations
n = 100 # number of steps
T = 1 # time to maturity
nu = 0.8 # vol of vol
S0 = 100 # stock price at T=0
K = 105 # strike price
r = 0 # interest rate
Sigma0 = 0.4 # starting vol
rho = 0 # correlation or not'

Black-Scholes Function

In [ ]:
def d1(S0, K, r, sigma, T):
    return (np.log(S0/K) + (r + sigma**2 / 2) * T)/(sigma * np.sqrt(T))
 
def d2(S0, K, r, sigma, T):
    return (np.log(S0 / K) + (r - sigma**2 / 2) * T) / (sigma * np.sqrt(T))
 
def BlackScholes(S0, K, r, sigma, T):
    return S0*ss.norm.cdf(d1(S0, K, r, sigma, T)) - K * np.exp(-r * T) * ss.norm.cdf(d2(S0, K, r, sigma, T))

**Pricing Vanilla Options using SABR Model**

Crude Monte Carlo Function

In [ ]:
def CrudeMC(h,m,n,T,nu,S0,K,r,Sigma0,rho):
    vMcSabr = np.zeros([h]) # vector to store mean of each sim
    for p in range(0,h):
        vPayoffs = np.zeros([m]) # vector to store vol of each sim
        vFinSk = np.zeros([m]) # vector to store final value of each sim
        for j in range (0,m):  # start loop doing 1000 sims
            vW = np.random.normal(0,1,n) # vector of random draws (0, 1)
            vZ = np.random.normal(0,1,n) # vector of random draws (0, 1)
            vSigma = np.zeros([n+1]) # vector to store vol
            vSk = np.zeros([n+1]) # vector to store vol
            vBM1 = np.zeros([n+1])
            vSigma[0] = Sigma0 # first element of vector
            vSk[0] = S0
            for i in range (0,n): # start loop for 100 steps
                vBM1[i+1] = vBM1[i]+vW[i]*np.sqrt(T/n)
                vSigma[i+1] = vSigma[0]*np.exp(nu*vBM1[i]-0.5*(nu**2)*T*(i+1)/n) # SABR sigma
                vSk[i+1] = vSk[i]+vSigma[i]*vSk[i]*(rho*vW[i]*np.sqrt(T/n)+np.sqrt(1-rho**2)*vZ[i]*np.sqrt(T/n)) #calculate stock prices
            vFinSk[j] = vSk[-1] # extract final observation of sim
            vPayoffs[j] = max(vFinSk[j]-K, 0) # payoff using final price of sim
        vMcSabr[p] = np.mean(vPayoffs)
    return(vMcSabr)

Conditional Monte Carlo Function

In [ ]:
# For rho != 0
def ConditionalMCRho(h,m,n,T,nu,S0,K,r,Sigma0,rho):
    vCMCBS = np.zeros([h])
    for p in range(0,h):
        vBS = np.zeros([m])
        for j in range(0,m):
            vW = np.random.normal(0,1,n)
            vBM = np.zeros([n+1])
            vSigma = np.zeros([n+1])
            vSigma[0] = Sigma0
            vBM[0] = 0
            iIntegral2 = 0
            for i in range(0, n):
                vBM[i+1] = vBM[i]+vW[i]*np.sqrt(T/n)
                vSigma[i+1] = vSigma[0]*np.exp((-0.5*nu**2)*(T*(i+1)/n)+nu*vBM[i+1])
                iIntegral2 = iIntegral2+vSigma[i]*(vW[i]*np.sqrt(T/n))
            iVol = np.sqrt(np.mean(vSigma**2))
            iIntegral = np.sum(vSigma**2)*T/n
            iS0Prime = S0*np.exp(-0.5*rho**2*iIntegral+rho*iIntegral2)
            iVol2 = np.sqrt(1-rho**2)*iVol
            iBS = BlackScholes(iS0Prime,K,r,iVol2,T)
            vBS[j] = iBS
        vCMCBS[p] = np.mean(vBS)
    return(vCMCBS)

In [ ]:
# For rho = 0
def ConditionalMC(h,m,n,T,nu,S0,K,r,Sigma0):
    vCmcSabr = np.zeros([h]) # vector to store mean of each sim
    for p in range(0,h):
        vCmcSabrSim = np.zeros([m]) # vector to store vol of each sim
        for j in range (0,m):  # start loop doing 1000 sims
            vW = np.random.normal(0,1,n) # vector of random draws (0, 1)
            vSigma = np.zeros([n+1]) # vector to store vol
            iIntegral = 0
            vBM = np.zeros([n+1])
            vSigma[0] = Sigma0 # first element of vector
            for i in range (0,n): # start loop for 100 steps
                vBM[i+1] = vBM[i]+vW[i]*np.sqrt(T/n) # only one Bmotion
                vSigma[i+1] = vSigma[0]*np.exp(nu*vBM[i]-0.5*(nu**2)*T*(i+1)/n) # SABR sigma
                iIntegral = iIntegral+(T/n)*((vSigma[i+1])**2) # calculate integral for whole sim
            vCmcSabrSim[j] = BlackScholes(S0, K, r, np.sqrt(iIntegral/T), T) # calculate BS price using SABR vol
        vCmcSabr[p] = np.mean(vCmcSabrSim)
    return(vCmcSabr)

Conditional Monte Caro with Antithetic Variates

In [ ]:
# For rho != 0
def AntCMCRho(h,m,n,T,nu,S0,K,r,Sigma0,rho):
    vCMCBS = np.zeros([h])
    vCMCBS_minus = np.zeros([h])
    vCMCA = np.zeros([h])
    for p in range(0,h):
        vBS = np.zeros([m])
        vBS_minus = np.zeros([m])
        for j in range(0,m):
            vW = np.random.normal(0,1,n)
            vW_minus = -vW
            vBM = np.zeros([n+1])
            vBMminus = np.zeros([n+1])
            vSigma = np.zeros([n+1])
            vSigma[0] = Sigma0
            vSigma_minus = np.zeros([n+1])
            iIntegral2 = 0
            iIntegral2_minus = 0
            for i in range(0, n):
                vBM[i+1] = vBM[i]+vW[i]*np.sqrt(T/n)
                vBMminus[i+1] = -vBM[i+1]
                vSigma[i+1] = vSigma[0]*np.exp((-0.5*nu**2)*(T*(i+1)/n)+nu*vBM[i+1])
                vSigma_minus[i+1] = vSigma[0]*np.exp(nu*vBMminus[i+1]-0.5*(nu**2)*T*(i+1)/n)
                iIntegral2 = iIntegral2+vSigma[i]*(vW[i]*np.sqrt(T/n))
                iIntegral2_minus = iIntegral2_minus+vSigma_minus[i]*(vW_minus[i]*np.sqrt(T/n))
            iVol = np.sqrt(np.mean(vSigma**2))
            iVol_minus = np.sqrt(np.mean(vSigma_minus**2))
            iIntegral = np.sum(vSigma**2)*T/n
            iIntegral_minus = np.sum(vSigma_minus**2)*T/n
            iS0Prime = S0*np.exp(-0.5*rho**2*iIntegral+rho*iIntegral2)
            iS0Prime_minus = S0*np.exp(-0.5*rho**2*iIntegral_minus+rho*iIntegral2_minus)
            iVol2 = np.sqrt(1-rho**2)*iVol
            iVol2_minus = np.sqrt(1-rho**2)*iVol_minus
            iBS = BlackScholes(iS0Prime,K,r,iVol2,T)
            iBS_minus = BlackScholes(iS0Prime_minus,K,r,iVol2_minus,T)
            vBS[j] = iBS
            vBS_minus[j] = iBS_minus
        vCMCBS[p] = np.mean(vBS)
        vCMCBS_minus[p] = np.mean(vBS_minus)
        vCMCA[p] = (vCMCBS[p]+vCMCBS_minus[p])/2
    return(vCMCA)

In [ ]:
# For rho = 0
def AntCMC(h,m,n,T,nu,S0,K,r,Sigma0):
    vCmcSabr = np.zeros([h])
    vCmcSabr_minus = np.zeros([h])# vector to store mean of each sim
    vCMCA  = np.zeros([h])
    for p in range(0,h):
        vCmcSabrSim = np.zeros([m])
        vCmcSabrSim_minus = np.zeros([m]) # vector to store vol of each sim
        for j in range (0,m):  # start loop doing 1000 sims
            vW = np.random.normal(0,1,n) # vector of random draws (0, 1)
            vSigma = np.zeros([n+1]) # vector to store vol
            vSigma_minus = np.zeros([n+1])
            iIntegral = 0
            iIntegral_minus = 0
            vBM = np.zeros([n+1])
            vBMminus = np.zeros([n+1])
            vSigma[0] = Sigma0 # first element of vector
            for i in range (0,n): # start loop for 100 steps
                vBM[i+1] = vBM[i]+vW[i]*np.sqrt(T/n) # only one Bmotion
                vBMminus[i+1] = -vBM[i+1]
                vSigma[i+1] = vSigma[0]*np.exp(nu*vBM[i]-0.5*(nu**2)*T*(i+1)/n) # SABR sigma
                vSigma_minus[i+1] = vSigma[0]*np.exp(nu*vBMminus[i]-0.5*(nu**2)*T*(i+1)/n)
                iIntegral = iIntegral+(T/n)*((vSigma[i+1])**2)
                iIntegral_minus = iIntegral_minus+(T/n)*((vSigma_minus[i+1])**2) # calculate integral for whole sim
            vCmcSabrSim[j] = BlackScholes(S0, K, r, np.sqrt(iIntegral/T), T)
            vCmcSabrSim_minus[j] = BlackScholes(S0, K, r, np.sqrt(iIntegral_minus/T), T) # calculate BS price using SABR vol
        vCmcSabr[p] = np.mean(vCmcSabrSim)
        vCmcSabr_minus[p] = np.mean(vCmcSabrSim_minus)
        vCMCA[p] = (vCmcSabr[p]+vCmcSabr_minus[p])/2
    return(vCMCA)

Crude Monte Carlo with Control Variates

In [ ]:
def CrudeMCCVar(h,m,n,T,nu,S0,K,r,Sigma0,rho):
    vMCControl = np.zeros([h]) # vector to store mean of each sim
    for p in range(0,h):
        vPayoffs = np.zeros([m]) # vector to store vol of each sim
        vFinSk = np.zeros([m]) # vector to store final value of each sim
        for j in range (0,m):  # start loop doing 1000 sims
            vW = np.random.normal(0,1,n) # vector of random draws (0, 1)
            vZ = np.random.normal(0,1,n) # vector of random draws (0, 1)
            vSigma = np.zeros([n+1]) # vector to store vol
            vSk = np.zeros([n+1]) # vector to store vol
            vBM1 = np.zeros([n+1])
            vSigma[0] = Sigma0 # first element of vector
            vSk[0] = S0
            for i in range (0,n): # start loop for 100 steps
                vBM1[i+1] = vBM1[i]+vW[i]*np.sqrt(T/n)
                vSigma[i+1] = vSigma[0]*np.exp(nu*vBM1[i]-0.5*(nu**2)*T*(i+1)/n) # SABR sigma
                vSk[i+1] = vSk[i]+vSigma[i]*vSk[i]*(rho*vW[i]*np.sqrt(T/n)+np.sqrt(1-rho**2)*np.sqrt(T/n)*vZ[i]) #calculate stock prices
            vFinSk[j] = vSk[-1]
            vPayoffs[j] = max(vFinSk[j]-K, 0) # payoff using final price of sim
            iCov = -np.cov(vPayoffs,vFinSk)[0][1]/np.var(vFinSk)
        vMCControl[p] = np.mean(vPayoffs+iCov*(vFinSk - S0))
    return(vMCControl)

Create Dataframe to Store Results

In [ ]:
# Rho = 0 

# Crude MC
starttime = time.time()
CrudeMC = CrudeMC(h,m,n,T,nu,S0,K,r,Sigma0,0)
totaltimeCrude = round((time.time() - starttime), 2)

# Conditional MC
starttime = time.time()
CondiMC = ConditionalMC(h,m,n,T,nu,S0,K,r,Sigma0)
totaltimeCondi = round((time.time() - starttime), 2)

# Antithetic CondiMC
starttime = time.time()
AntCMC = AntCMC(h,m,n,T,nu,S0,K,r,Sigma0)
totaltimeAnt = round((time.time() - starttime), 2)

# Control Variate CrudeMC
starttime = time.time()
CrudeMCCVar = CrudeMCCVar(h,m,n,T,nu,S0,K,r,Sigma0,0)
totaltimeCrudeMCCVar = round((time.time() - starttime), 2)

data = [[np.mean(CrudeMC), np.mean(CondiMC), np.mean(AntCMC), np.mean(CrudeMCCVar)], 
        [np.std(CrudeMC), np.std(CondiMC), np.std(AntCMC), np.std(CrudeMCCVar)], 
        [totaltimeCrude, totaltimeCondi, totaltimeAnt, totaltimeCrudeMCCVar]]
dfZero = pd.DataFrame(data, columns=['CrudeMC', 'ConditionalMC', 'AntitheticCMC', 'ControlVarMC'], 
                  index=['Mean','SD', 'Time'])
dfZero


In [ ]:
# Rho != 0

# Crude MC
starttime = time.time()
CrudeMC = CrudeMC(h,m,n,T,nu,S0,K,r,Sigma0,rho)
totaltimeCrude = round((time.time() - starttime), 2)

# Conditional MC
starttime = time.time()
CondiMC = ConditionalMCRho(h,m,n,T,nu,S0,K,r,Sigma0,rho)
totaltimeCondi = round((time.time() - starttime), 2)

# Antithetic CMC
starttime = time.time()
AntCMC = AntCMCRho(h,m,n,T,nu,S0,K,r,Sigma0,rho)
totaltimeAnt = round((time.time() - starttime), 2)

# Control Variate CrudeMC
starttime = time.time()
CrudeMCCVar = CrudeMCCVar(h,m,n,T,nu,S0,K,r,Sigma0,rho)
totaltimeCrudeMCCVar = round((time.time() - starttime), 2)

data = [[np.mean(CrudeMC), np.mean(CondiMC), np.mean(AntCMC), np.mean(CrudeMCCVar)], 
        [np.std(CrudeMC), np.std(CondiMC), np.std(AntCMC), np.std(CrudeMCCVar)], 
        [totaltimeCrude, totaltimeCondi, totaltimeAnt, totaltimeCrudeMCCVar]]
df = pd.DataFrame(data, columns=['CrudeMC', 'ConditionalMC', 'AntitheticCMC', 'ControlVarMC'], 
                  index=['Mean','SD', 'Time'])
df


**Pricing Fixed Strike Arithmetic Asian Options**

Crude Monte Carlo

In [ ]:
def AsianOptCrude(h,m,n,T,S0,K,r,Sigma0):
    vMeanPayoffs = np.zeros([h])
    for p in range(0,h):
        mSt = np.zeros([m,n+1])
        # Simulate stock prices m times for n steps
        for j in range(0,m):
            mSt[j,0] = S0
            vZ = np.random.normal(0,1,n)
            for i in range(0,n):
                mSt[j,i+1] = mSt[j,i]+mSt[j,i]*Sigma0*vZ[i]*np.sqrt(T/n)
        # Calculate mean and payoff of each sim
        vMeanSim = np.zeros([m])
        vPayoffs = np.zeros([m])
        for j in range(0,m):
            vMeanSim[j] = np.mean(mSt[j,])
            vPayoffs[j] = max(vMeanSim[j]-K, 0)
        # Take mean of all payoffs
        vMeanPayoffs[p] = np.mean(vPayoffs)     
    return(vMeanPayoffs)

Crude Monte Carlo with Antithetic Variates 

In [ ]:
def AsianOptAntithetic(h,m,n,T,S0,K,r,Sigma0):
    dMeanPayoffs = np.zeros([h])
    dMeanPayoffs_minus = np.zeros([h])
    dAsianAntithetic = np.zeros([h])
    for p in range(0,h):
        mSt = np.zeros([m,n+1])
        mSt_minus = np.zeros([m,n+1])
        # Simulate stock prices m times for n steps
        for j in range(0,m):
            mSt[j,0] = S0
            mSt_minus[j,0] = S0
            vZ = np.random.normal(0,1,n)
            vZ_minus = -vZ
            for i in range(0,n):
                mSt[j,i+1] = mSt[j,i]+mSt[j,i]*Sigma0*vZ[i]*np.sqrt(T/n)
                mSt_minus[j,i+1] = mSt_minus[j,i]+mSt_minus[j,i]*Sigma0*vZ_minus[i]*np.sqrt(T/n)
        # Calculate mean and payoff of each sim
        vMeanSim = np.zeros([m])
        vPayoffs = np.zeros([m])
        vMeanSim_minus = np.zeros([m])
        vPayoffs_minus = np.zeros([m])
        for j in range(0,m):
            vMeanSim[j] = np.mean(mSt[j,])
            vMeanSim_minus[j] = np.mean(mSt_minus[j,])
            vPayoffs[j] = max(vMeanSim[j]-K, 0)
            vPayoffs_minus[j] = max(vMeanSim_minus[j]-K, 0)
        # Take mean of all payoffs
        dMeanPayoffs[p] = np.mean(vPayoffs)
        dMeanPayoffs_minus[p] = np.mean(vPayoffs_minus)
        dAsianAntithetic[p] =  (dMeanPayoffs[p] + dMeanPayoffs_minus[p])/2 
    return(dAsianAntithetic)

Crude Monte Carlo with Control Variates

In [ ]:
def AsianOptControl(h,m,n,T,S0,K,r,Sigma0):
    vAsianMCControl = np.zeros([h])
    SigmaA = Sigma0/np.sqrt(3)
    d1 = (np.log(S0/K)+((1/2)*SigmaA)*T)/(SigmaA*T)
    d2 = d1 - SigmaA*T
    iGt = S0*ss.norm.cdf(d1) - K*ss.norm.cdf(d2) #E(Z)
    for p in range(0,h):
        mSt = np.zeros([m,n+1])
        # Simulate stock prices m times for n steps
        for j in range(0,m):
            mSt[j,0] = S0
            vZ = np.random.normal(0,1,n)
            for i in range(0,n):
                mSt[j,i+1] = mSt[j,i]+mSt[j,i]*Sigma0*vZ[i]*np.sqrt(T/n)
        vGeoMean =np.zeros([m])
        vGeoPayoffs = np.zeros([m])
        vArithMean = np.zeros([m])
        vArithPayoffs = np.zeros([m])
        for j in range(0,m):
            vGeoMean[j] = ss.gmean(mSt[j,])
            vGeoPayoffs[j] = max(vGeoMean[j]-K, 0) #Z
            vArithMean[j] = np.mean(mSt[j,])
            vArithPayoffs[j] = max(vArithMean[j]-K, 0) #Y
        iCov = -np.cov(vArithPayoffs,vGeoPayoffs)[0][1]/np.var(vGeoPayoffs)
        vAsianMCControl[p] = np.mean(vArithPayoffs+iCov*(vGeoPayoffs - iGt))
    return(vAsianMCControl)

Create Dataframe to Store Results

In [ ]:
starttime = time.time()
AsianOptCrude = AsianOptCrude(50,100,100,1,100,80,0,0.4)
totaltimeAsianOptCrude = round((time.time() - starttime), 2)

starttime = time.time()
AsianOptAntithetic = AsianOptAntithetic(50,100,100,1,100,80,0,0.4)
totaltimeAsianOptAntithetic = round((time.time() - starttime), 2)

starttime = time.time()
AsianOptControl = AsianOptControl(50,100,100,1,100,80,0,0.4)
totaltimeAsianOptControl = round((time.time() - starttime), 2)

data = [[np.mean(AsianOptCrude), np.mean(AsianOptAntithetic), np.mean(AsianOptControl)], 
        [np.std(AsianOptCrude), np.std(AsianOptAntithetic), np.std(AsianOptControl)], 
        [totaltimeAsianOptCrude, totaltimeAsianOptAntithetic, totaltimeAsianOptControl]]
df = pd.DataFrame(data, columns=['Crude', 'Antithetic Variate', 'Control Variate'], 
                  index=['Mean','SD', 'Time'])
df